In [1]:
from pathlib import Path 
import os 

In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_data(file_path):
    """Read data from file into two vectors: timestamps and durations."""
    data = pd.read_csv(file_path, sep='\t', header=None, names=['timestamps', 'durations'])
    return data['timestamps'].values, data['durations'].values

def generate_bin(durations, interval):
    """
    Bin durations into intervals and calculate density.
    
    Args:
        durations: List of duration values
        interval: Size of each bin
        
    Returns:
        bin_centers: Centers of each bin
        bin_density: Density count in each bin
    """
    max_duration = np.max(durations)
    num_bins = int(np.ceil(max_duration / interval))
    bins = np.linspace(0, num_bins * interval, num_bins + 1)
    
    hist, bin_edges = np.histogram(durations, bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_density = hist / (len(durations) * interval)
    
    return bin_centers, bin_density

def kde_traces(durations, interval, name=None):
    """
    Generate histogram and density estimate traces.
    
    Args:
        durations: List of duration values
        interval: Size of each bin
        name: Legend group name (defaults to interval+μs if None)
    
    Returns:
        List of plotly traces for histogram and density
    """
    bin_centers, bin_density = generate_bin(durations, interval)
    
    # Set default legend group name if not provided
    if name is None:
        name = f"{interval}μs"
    
    # Generate a consistent color for both traces
    color = f"hsl({hash(name) % 360}, 70%, 50%)"
    
    # Create histogram trace
    histogram_trace = go.Histogram(
        x=durations,
        xbins=dict(
            start=0,
            end=np.max(durations),
            size=interval
        ),
        histnorm='probability density',
        name=f"{name} histogram",
        legendgroup=name,
        opacity=0.7,
        marker_color=color
    )
    
    # Create density trace
    density_trace = go.Scatter(
        x=bin_centers,
        y=bin_density,
        mode='markers+lines',
        name=f"{name} density",
        legendgroup=name,
        marker=dict(size=8, color=color),
        line=dict(color=color)
    )
    
    return [density_trace]

In [13]:
data_dir = Path('../raw_data/life_muon.txt')
timestamps, durations = read_data(data_dir)

traces = []
for i in [0.05, 0.1, 0.2, 0.5, 1]: 
    traces.extend(kde_traces(durations, i))

fig = go.Figure(data=traces)
fig.show()